In [1]:
!pip install git+https://github.com/jyucsiro/loci-scripts.git@feat-mar2020-cache
!pip install numpy pandas

  Cloning https://github.com/jyucsiro/loci-scripts.git (to revision feat-mar2020-cache) to /tmp/pip-req-build-0fqmq_ls
  Running command git clone -q https://github.com/jyucsiro/loci-scripts.git /tmp/pip-req-build-0fqmq_ls
  Running command git checkout -b feat-mar2020-cache --track origin/feat-mar2020-cache
  Switched to a new branch 'feat-mar2020-cache'
  Branch 'feat-mar2020-cache' set up to track remote branch 'feat-mar2020-cache' from 'origin'.
  Running command git submodule update --init --recursive -q
     |████████████████████████████████| 141 kB 796 kB/s eta 0:00:01
     |████████████████████████████████| 344 kB 2.0 MB/s eta 0:00:01
  Created wheel for pyloci: filename=pyloci-0.1.1.25-py3-none-any.whl size=34779 sha256=f5314d00ced08a652495badc8b43367da1b7cda41c8d2d61966008a6585ae51b
  Stored in directory: /tmp/pip-ephem-wheel-cache-5q9qe97o/wheels/a6/ce/e9/089abd8bae28888f40ab1b215bb310bb68c42e1cce6df9dfda
Successfully built pyloci
  Attempting uninstall: rdflib
    Found exi

# Reapportioning ASGS16 LGA to SA1  example using the pyloci API

This notebook show how you would perform reapportioning using the [pyloci](https://pypi.org/project/pyloci/) library using a simple CSV file. The pyloci library interfaces directly with the SPARQL API for the Loc-I Cache GraphDB.

In [29]:
import numpy as np
import pandas as pd
df = pd.read_csv('loci-lga-sa1-test-case1.csv', delimiter = ',')
#show the csv file read in
df

,LGA,Count
0,http://linked.data.gov.au/dataset/asgs2016/loc...,100
1,http://linked.data.gov.au/dataset/asgs2016/loc...,200


In [30]:
#sure the config for the SPARQL endpoint to hit is set
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
print(os.getenv("SPARQL_ENDPOINT"))

http://db.loci.cat/repositories/loci-cache


In [31]:
#import the reapportioning module from pyloci
from pyloci import reapportioning


from pyloci.api.util import Util as API_Util
from pyloci.sparql import util as sparql_util
import time

auth = None
api_util = API_Util()


In [32]:
toFeatureType = "http://linked.data.gov.au/def/asgs#StatisticalAreaLevel1"
LOCI_INTEGRATION_API = "https://test-api.loci.cat/api/v1"

In [39]:
res_df_cols = {
    "LGA" : [],
    "SA1" : [],
    "percent_overlap": [],
    "source_data": [],
    "reapportioned_data": [],
}
for index, row in df.iterrows():
    fromFeature = row['LGA']
    print(row['LGA'], row['Count'])
    print("Querying overlaps of {} to {}...".format(fromFeature, toFeatureType))
    tic = time.perf_counter()
    list_locations = api_util.query_api_location_overlaps(fromFeature, toFeatureType, LOCI_INTEGRATION_API, crosswalk='true')
    toc = time.perf_counter()
    print(f"query_api_location_overlaps took {toc - tic:0.4f} seconds")
    #print(list_locations)
    for o in list_locations['overlaps']:
        res_df_cols["LGA"].append(fromFeature)
        res_df_cols["SA1"].append(o['uri'])
        res_df_cols["percent_overlap"].append(o['forwardPercentage'])
        res_df_cols["source_data"].append(row['Count'])
        reapportioned = (float(o['forwardPercentage'])/100.0)*float(row['Count'])
        res_df_cols["reapportioned_data"].append(reapportioned)

        

http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/44000 100
Querying overlaps of http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/44000 to http://linked.data.gov.au/def/asgs#StatisticalAreaLevel1...
query_api_location_overlaps took 0.4090 seconds
http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/55880 200
Querying overlaps of http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/55880 to http://linked.data.gov.au/def/asgs#StatisticalAreaLevel1...
query_api_location_overlaps took 2.2934 seconds


In [40]:
res_df = pd.DataFrame (res_df_cols, columns = ['LGA', "SA1", "percent_overlap", "source_data", "reapportioned_data"])
res_df

,LGA,SA1,percent_overlap,source_data,reapportioned_data
0,http://linked.data.gov.au/dataset/asgs2016/loc...,http://linked.data.gov.au/dataset/asgs2016/sta...,0.006331,100,0.006331
1,http://linked.data.gov.au/dataset/asgs2016/loc...,http://linked.data.gov.au/dataset/asgs2016/sta...,99.993669,100,99.993669
2,http://linked.data.gov.au/dataset/asgs2016/loc...,http://linked.data.gov.au/dataset/asgs2016/sta...,20.173726,200,40.347453
3,http://linked.data.gov.au/dataset/asgs2016/loc...,http://linked.data.gov.au/dataset/asgs2016/sta...,79.826275,200,159.652550
